In [9]:
%%time

import numpy as np
import os
import matplotlib.pyplot as plt

import reconstruction

import keras

from sklearn.metrics import mean_squared_error
from skimage.metrics import structural_similarity
from skimage.measure.entropy import shannon_entropy

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
def createNeighborhoodVector (pic: np.ndarray, coordinates: tuple) -> list:
    m, n = coordinates[0], coordinates [1]
    return           [pic [m, n-1], 
                      pic [m-1, n-1],
                      pic [m-1, n],
                      pic [m-1, n+1]]

def convertPicInt8ToFloat32 (pic: np.ndarray) -> np.ndarray:
    return np.array (pic, dtype='float32')

def normalizePic (pic: np.ndarray) -> np.ndarray:
    return pic / 255.0;

def createArrayofNeighborhoodVectors (pic: np.ndarray) -> np.ndarray:
    temp = []

    for i in range (pic[1:, 1:-1].shape [0]):
        for j in range (pic[1:, 1:-1].shape [1]):
            temp.append (createNeighborhoodVector (pic, (i, j)))

    neighborhood_vectors_array = np.array (temp, dtype='float32')

    return neighborhood_vectors_array


In [11]:
model = keras.models.load_model ('saved_prediction_model.keras')

In [12]:
df = reconstruction.convertExcel2Dataframe ('ACoefs.xlsx')

df.aCoefs_optimized [2]

'[0.29421388 0.15952316 0.29602125 0.25434889]'

In [ ]:
%%time

pic_path = 'test_pics/Downloaded Persian Miniatures - Cropped and Resized' + '/'

for i, pic_name in enumerate(os.listdir (pic_path)):
    pic = reconstruction.createPicArrayFromPath (pic_path + pic_name, '')

    y_values = np.array (pic[1:, 1:-1].ravel (), dtype='float32') / 255.0
    neighborhood_vectors_array = createArrayofNeighborhoodVectors (pic) / 255.0


    model = keras.models.Sequential ()
    model.add (keras.layers.Input ((4,)))
    model.add (keras.layers.Dense (6, activation='tanh'))
    model.add (keras.layers.Dense (3, activation='tanh'))
    model.add (keras.layers.Dense (1, activation='sigmoid'))

    model.compile (optimizer='adam', loss = 'mean_squared_error')

    model.fit (neighborhood_vectors_array, y_values, epochs = 10)


    prediction = model.predict (neighborhood_vectors_array)

    reconstructed = (prediction * 255.0).reshape (pic [1:, 1:-1].shape)

    diff_matrix = reconstruction.createDiffMatrixFromMatrices (pic [1:, 1:-1], reconstructed)

    mse_score = mean_squared_error (pic [1:, 1:-1], reconstructed)
    ssim_score = structural_similarity (pic [1:, 1:-1], np.array (reconstructed, dtype='uint8'))
    entropy_score = shannon_entropy (diff_matrix)

    linear_reconstructed = reconstruction.reconstructPicfromACoefs (reconstruction.acoef_str2array (df.aCoefs_optimized [i]), pic)

    linear_mse_score = mean_squared_error (pic, linear_reconstructed)
    linear_ssim_score = structural_similarity (pic , np.array (linear_reconstructed, dtype='uint8'))
    linear_entropy_score = shannon_entropy (reconstruction.createDiffMatrixFromMatrices (pic, linear_reconstructed))


    plt.figure (figsize=(20,15))

    plt.subplot (1,3,1)
    plt.imshow (linear_reconstructed, cmap='gray')
    plt.title ('Linear Prediction')
    plt.text (0, 600, 'MSE Score: {}\nSSIM Score: {}\nEntropy Score: {}'.format(linear_mse_score, linear_ssim_score, linear_entropy_score))

    plt.subplot (1,3,2)
    plt.imshow (np.array (reconstructed, dtype='uint8'), cmap='gray')
    plt.title ('MLP Prediction')
    plt.text (0, 600, 'MSE Score: {}\nSSIM Score: {}\nEntropy Score: {}'.format(mse_score, ssim_score, entropy_score))

    plt.subplot (1,3,3)
    plt.imshow (pic [1:,1:-1], cmap='gray')
    plt.title ('Original Picture')

    plt.savefig ('report/' + pic_name)
    plt.close ()


Epoch 1/10
8145/8145 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - loss: 0.0165
Epoch 2/10
8145/8145 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - loss: 0.0105
Epoch 3/10
8145/8145 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - loss: 0.0104
Epoch 4/10
8145/8145 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.0104
Epoch 5/10
3401/8145 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0105

In [ ]:
pic_path = 'test_pics/Downloaded Persian Miniatures - Cropped and Resized' + '/'


for i, pic_name in enumerate(os.listdir (pic_path)):
    print (i, pic_name)

0 103-La alusion, Miniatura de Ostad Hosein Behzad, Museo Behzad, 1964.jpg
1 105-Ferdowsi; Construi un alto castillo de versos Miniatura de Ostad Hosein Behzad.jpg
2 109-El alfarero de la existencia, Miniatura de Ostad Hosein Behzad.jpg
3 115Poema de Hafiz Sirveme una copa que traen las huries del Paraiso, Miniatura de Ostad Hosein Behzad.jpg
4 116-Khayam, Miniatura de Ostad Hosein Behzad.jpg
5 119Khayam, Miniatura de Ostad Hosein Behzad.jpg
6 121-Rudaki, Miniatura de Ostad Hosein Behzad.jpg
7 122-Los que dominan el conocimiento y las artes, Miniatura de Ostad Hosein Behzad.jpg
8 126-La creacion de Dios, Miniatura de Ostad Hosein Behzad.jpg
9 127-Rostam y Sohrab, Miniatura de Ostad Hosein Behzad.jpg
10 138-Ay, la juventud ha llegado a su fin, Miniatura de Ostad Hosein Behzad.jpg
11 141-Shams Tabrizi, Miniatura de Ostad Hosein Behzad.jpg
12 142-Rudaki, Miniatura de Ostad Hosein Behzad.jpg
13 144-el_nacimiento_de_jesus_miniatura_de_ostad_hosein_behzad.jpg
14 149-sheij_attar_miniatura_de_